In [1]:
import treecorr
import pyccl as ccl
import numpy as np
    
config = {'ra_col' : 1,
          'dec_col' : 2,
          'g1_col' : 3,
          'g2_col' : 4,
          'w_col' : 5,
          'ra_units' : 'deg',
          'dec_units' : 'deg'}

We load all of the data into a single array for simplicity, n=6 is used for consistency so that we can start counting from 1 to 5 instead of from 0 to 4

Here we calculate the auto correlation functions for all bins

In [2]:
for i in np.arange(5)+1:
    filename = '/global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/GalCat_tomo{}_All_0.06GpAM_RA_Dec_g1_g2_w_den_Sij.dat'.format(i)
    cat = treecorr.Catalog(filename, config, flip_g2=True)
    #gg = treecorr.GGCorrelation(nbins = 20, min_sep = 0.005, max_sep = 7, bin_slop=0.01) 
    gg = treecorr.GGCorrelation(nbins = 20, min_sep=0.5, max_sep=475.5, sep_units='arcmin',bin_slop=0.01,)
    gg.process(cat, metric = 'Arc')
    gg.write('correlations/cross_correlation_tomo{}_{}'.format(i,i))

Here we calculate the cross correlation functions for all bins

In [3]:
for i in np.arange(5)+1:
    for j in range(1, i):
        filename1 = '/global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/GalCat_tomo{}_All_0.06GpAM_RA_Dec_g1_g2_w_den_Sij.dat'.format(i)
        filename2 = '/global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/GalCat_tomo{}_All_0.06GpAM_RA_Dec_g1_g2_w_den_Sij.dat'.format(j)
        cat_1 = treecorr.Catalog(filename1, config, flip_g2='True')
        cat_2 = treecorr.Catalog(filename2, config, flip_g2='True')
        #gg = treecorr.GGCorrelation(nbins = 20, min_sep = 0.005, max_sep = 7, bin_slop=0.01) 
        gg = treecorr.GGCorrelation(nbins = 20, min_sep=0.5, max_sep=475.5, sep_units='arcmin',bin_slop=0.01)
        gg.process(cat_1, cat_2)
        #gg.process(cat_1, cat_2, metric = 'Arc')
        gg.write('correlations/cross_correlation_tomo{}_{}'.format(j,i))

Here we show how to read the correlation functions saved to a file by TreeCorr

In [4]:
gg = treecorr.GGCorrelation(nbins = 20,
                       min_sep = 0.03333333, #2 arcmin
                       max_sep = 1.66666667)
gg.read('correlations/cross_correlation_tomo3_3')

In [5]:
from astropy.table import Table

In [10]:

##'ra','dec','z','weight','delta','e_obs_1','e_obs_2', 'e_obs_no_noise_1','e_obs_no_noise_2','e_IA_TATT_1', 'e_IA_TATT_2'

# Run the auto-correlation functions:
config_IA = {'ra_col' : 1,
          'dec_col' : 2,
          'g1_col' : 6,
          'g2_col' : 7,
          'w_col' : 4,
          'ra_units' : 'deg',
          'dec_units' : 'deg'}


# Run the auto-correlation functions:
for i in np.arange(5)+1:
    filename = '/global/cscratch1/sd/lhior/fin_epsilon_IA_{}.dat'.format(i)
    cat = treecorr.Catalog(filename, config_IA, flip_g2=True,first_row=2)
    #gg = treecorr.GGCorrelation(nbins = 20, min_sep = 0.005, max_sep = 7, bin_slop=0.01) 
    gg = treecorr.GGCorrelation(nbins = 20, min_sep=0.5, max_sep=475.5, sep_units='arcmin',bin_slop=0.01,)
    gg.process(cat, metric = 'Arc')
    gg.write('correlations/infused_cross_correlation_tomo{}_{}'.format(i,i))

# And the cross-redshift bins
for i in np.arange(5)+1:
    for j in range(1, i):
        filename1 = '/global/cscratch1/sd/lhior/fin_epsilon_IA_{}.dat'.format(i)
        filename2 = '/global/cscratch1/sd/lhior/fin_epsilon_IA_{}.dat'.format(j)
        cat_1 = treecorr.Catalog(filename1, config_IA, flip_g2='True',first_row=2)
        cat_2 = treecorr.Catalog(filename2, config_IA, flip_g2='True',first_row=2)
        #gg = treecorr.GGCorrelation(nbins = 20, min_sep = 0.005, max_sep = 7, bin_slop=0.01) 
        gg = treecorr.GGCorrelation(nbins = 20, min_sep=0.5, max_sep=475.5, sep_units='arcmin',bin_slop=0.01)
        gg.process(cat_1, cat_2)
        #gg.process(cat_1, cat_2, metric = 'Arc')
        gg.write('correlations/infused_cross_correlation_tomo{}_{}'.format(j,i))

In [ ]:
# Run treecorr on the mocks with IA infused, no noise:

# Run the auto-correlation functions:
config_IA = {'ra_col' : 1,
          'dec_col' : 2,
          'g1_col' : 8,
          'g2_col' : 9,
          'w_col' : 4,
          'ra_units' : 'deg',
          'dec_units' : 'deg'}


# Run the auto-correlation functions:
for i in np.arange(5)+1:
    filename = '/global/cscratch1/sd/lhior/fin_epsilon_IA_{}.dat'.format(i)
    cat = treecorr.Catalog(filename, config_IA, flip_g2=True,first_row=2)
    #gg = treecorr.GGCorrelation(nbins = 20, min_sep = 0.005, max_sep = 7, bin_slop=0.01) 
    gg = treecorr.GGCorrelation(nbins = 20, min_sep=0.5, max_sep=475.5, sep_units='arcmin',bin_slop=0.01,)
    gg.process(cat, metric = 'Arc')
    gg.write('correlations/infused_cross_correlation_no_noise_tomo{}_{}'.format(i,i))

# And the cross-redshift bins
for i in np.arange(5)+1:
    for j in range(1, i):
        filename1 = '/global/cscratch1/sd/lhior/fin_epsilon_IA_{}.dat'.format(i)
        filename2 = '/global/cscratch1/sd/lhior/fin_epsilon_IA_{}.dat'.format(j)
        cat_1 = treecorr.Catalog(filename1, config_IA, flip_g2='True',first_row=2)
        cat_2 = treecorr.Catalog(filename2, config_IA, flip_g2='True',first_row=2)
        #gg = treecorr.GGCorrelation(nbins = 20, min_sep = 0.005, max_sep = 7, bin_slop=0.01) 
        gg = treecorr.GGCorrelation(nbins = 20, min_sep=0.5, max_sep=475.5, sep_units='arcmin',bin_slop=0.01)
        gg.process(cat_1, cat_2)
        #gg.process(cat_1, cat_2, metric = 'Arc')
        gg.write('correlations/infused_cross_correlation_no_noise_tomo{}_{}'.format(j,i))

In [ ]:
# Run treecorr on the mocks with IA_TATT infused:


# Run the auto-correlation functions:
config_IA = {'ra_col' : 1,
          'dec_col' : 2,
          'g1_col' : 10,
          'g2_col' : 11,
          'w_col' : 4,
          'ra_units' : 'deg',
          'dec_units' : 'deg'}


# Run the auto-correlation functions:
for i in np.arange(5)+1:
    filename = '/global/cscratch1/sd/lhior/fin_epsilon_IA_{}.dat'.format(i)
    cat = treecorr.Catalog(filename, config_IA, flip_g2=True,first_row=2)
    #gg = treecorr.GGCorrelation(nbins = 20, min_sep = 0.005, max_sep = 7, bin_slop=0.01) 
    gg = treecorr.GGCorrelation(nbins = 20, min_sep=0.5, max_sep=475.5, sep_units='arcmin',bin_slop=0.01,)
    gg.process(cat, metric = 'Arc')
    gg.write('correlations/infused_cross_correlation_TATT_tomo{}_{}'.format(i,i))

# And the cross-redshift bins
for i in np.arange(5)+1:
    for j in range(1, i):
        filename1 = '/global/cscratch1/sd/lhior/fin_epsilon_IA_{}.dat'.format(i)
        filename2 = '/global/cscratch1/sd/lhior/fin_epsilon_IA_{}.dat'.format(j)
        cat_1 = treecorr.Catalog(filename1, config_IA, flip_g2='True',first_row=2)
        cat_2 = treecorr.Catalog(filename2, config_IA, flip_g2='True',first_row=2)
        #gg = treecorr.GGCorrelation(nbins = 20, min_sep = 0.005, max_sep = 7, bin_slop=0.01) 
        gg = treecorr.GGCorrelation(nbins = 20, min_sep=0.5, max_sep=475.5, sep_units='arcmin',bin_slop=0.01)
        gg.process(cat_1, cat_2)
        #gg.process(cat_1, cat_2, metric = 'Arc')
        gg.write('correlations/infused_cross_correlation_TATT_tomo{}_{}'.format(j,i))